## CUDA

In [1]:
import sys
print(sys.executable)
!{sys.executable} --version

/home/filip/Documents/.venvs/gnn/bin/python
Python 3.12.7


In [2]:
!/usr/local/cuda-12.4/bin/nvcc --version
print("Nvidia drivers: ", end="")
!nvidia-smi --query-gpu=driver_version --format=csv,noheader

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:18:24_PDT_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0
Nvidia drivers: 550.107.02


## Install PyTorch

In [ ]:
# https://pytorch.org/get-started/locally/
!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [4]:
import torch, torchvision, torchaudio

print("torch: ", torch.__version__)
print("torchaudio: ", torchvision.__version__)
print("torchvision: ", torchaudio.__version__)

torch:  2.4.1+cu124
torchaudio:  0.19.1+cu124
torchvision:  2.4.1+cu124


In [5]:
print(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

cuda


## Install PyTorch Geometric

In [ ]:
# https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html
!{sys.executable} -m pip install torch_geometric
!{sys.executable} -m pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu124.html

In [7]:
import torch_geometric
import torch_scatter
import torch_sparse
import torch_cluster
import torch_spline_conv

print("torch_geometric: ", torch_geometric.__version__)
print("torch_scatter version:", torch_scatter.__version__)
print("torch_sparse version:", torch_sparse.__version__)
print("torch_cluster version:", torch_cluster.__version__)
print("torch_spline_conv version:", torch_spline_conv.__version__)

torch_geometric:  2.6.1
torch_scatter version: 2.1.2+pt24cu124
torch_sparse version: 0.6.18+pt24cu124
torch_cluster version: 1.6.3+pt24cu124
torch_spline_conv version: 1.2.2+pt24cu124


## Test installation

In [9]:
from torch_geometric.data import Data
import torch_geometric.nn as pyg_nn

# Create a simple graph data example
edge_index = torch.tensor([[0, 1, 1], [1, 0, 2]], dtype=torch.long)
x = torch.tensor([[1, 2], [2, 3], [3, 4]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)

# Move data to the specified device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
print("Data device:", data.x.device)

# Define a simple GCN model
class SimpleGNN(torch.nn.Module):
    def __init__(self):
        super(SimpleGNN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(2, 2)
        self.conv2 = pyg_nn.GCNConv(2, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize and move the model to the device
model = SimpleGNN().to(device)
print("Model device:", next(model.parameters()).device)

# Perform a forward pass
output = model(data)
print("Model output:", output)

Data device: cuda:0
Model device: cuda:0
Model output: tensor([[-1.1759,  1.2075],
        [-1.1759,  1.2075],
        [-1.2063,  1.2387]], device='cuda:0', grad_fn=<AddBackward0>)
